<a href="https://colab.research.google.com/github/surendrallam/Caffe_Mobilenet_SSD_Custom_Dataset_Training/blob/main/Caffe_Mobilenet_SSD_Custom_Dataset_Training_on_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caffe Installation

In [ ]:
# !sudo apt install caffe-cuda
# !sudo apt build-dep caffe-cuda
!uname -a

Linux cb6a35eecea3 5.4.104+ #1 SMP Sat Jun 5 09:50:34 PDT 2021 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
!nvidia-smi
!nvcc --version

Fri Aug  6 09:08:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## Installing necessary dependencies.


In [ ]:
!apt-get -y install -y --no-install-recommends libboost-all-dev
!apt-get -y install libprotobuf-dev libleveldb-dev libsnappy-dev libopencv-dev \
libboost-all-dev libgoogle-glog-dev liblmdb-dev protobuf-compiler 

!apt-get -y install libgflags2.2 libgflags-dev 
!apt-get -y install libhdf5-100 libhdf5-serial-dev libhdf5-devlibhdf5-cpp-100


##  Cloning Caffe

In [ ]:
!git clone https://github.com/weiliu89/caffe.git
%cd caffe
!git checkout ssd
%cp Makefile.config.example Makefile.config

## Modifying Makefile

In [ ]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile 
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

## Installing necessary Python packages.

In [ ]:
!apt-get install python3-pip
!pip install scikit-image protobuf
%cd /content/caffe/python
!for req in $(cat requirements.txt); do pip install $req; done

In [ ]:
%cd /usr/lib/x86_64-linux-gnu/
# %ls -al | grep "libhdf5_serial"
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so.100.0.1 /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so.100.0.0 /usr/lib/x86_64-linux-gnu/libhdf5_hl.so
%env CPATH="/usr/include/hdf5/serial/"
!echo $CPATH

## Making Caffe

In [ ]:
%cd /content/caffe
# !make clean
!make all
!echo "Done Make all"
!make test
!echo "Done Make test"
!make runtest
!echo "Done Make runtest"

In [ ]:
!make test
!echo "Done Make test"

In [ ]:
!make runtest
!echo "Done Make runtest"

In [ ]:
!make pycaffe
!echo "Done Make pycaffe"

## Verifying Caffe

In [ ]:
# import cv2
# print(cv2.__version__)
import caffe

ModuleNotFoundError: ignored

# Mobilenet-SSD traning using Custom dataset 

## Cloning Mobilenet-SSD

In [ ]:
# %cd /content/
# !git clone https://github.com/chuanqi305/MobileNet-SSD.git
%cd /content/MobileNet-SSD/
# !ls

In [ ]:
%cd /content/MobileNet-SSD/
# !chmod +x train.sh
!./train.sh

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!echo "Mounted Google Drive"

In [ ]:
# Copying caffe directory from G-Drive
!cp -r /content/gdrive/MyDrive/caffe /content/

In [ ]:
# Copying caffe directory to G-Drive
!cp -r /content/caffe /content/MobileNet-SSD/ /content/gdrive/MyDrive/

In [ ]:
!cp -r caffe/ /content/gdrive/

# Reference:
* https://askubuntu.com/questions/960238/nvcc-fatal-unsupported-gpu-architecture-compute-20
* https://stackoverflow.com/questions/37007495/caffe-didnt-see-hdf5-h-when-compiling/44912342
* https://github.com/BVLC/caffe/issues/4621#issuecomment-243124779
* https://colab.research.google.com/github/Huxwell/caffe-colab/blob/main/caffe_details.ipynb#scrollTo=TpMhijZ6-9hy
* https://filip-drapejkowski.medium.com/caffe-in-google-colab-2021-2d730989c849